### This will copy images to folders

In [2]:
import face_recognition
import numpy as np
import os
import random
import json
import shutil

def organize_images(image_path, directory="assets/image_data/archive/push", paging_file="assets/image_data/archive/push/image_index.json"):
    """
    Organizes images into folders based on similarity to existing images,
    using face encodings for comparison. Handles multiple faces per image.

    Args:
        image_path (str): Path to the new image to be organized.
        directory (str): Base directory for storing images. 
                                     Defaults to "push".
        paging_file (str): Name of the JSON file to store 
                                      folder-encoding mappings. 
                                      Defaults to "image_index.json".
    """

    if not os.path.exists(directory):
        os.makedirs(directory)

    try:
        with open(paging_file, 'r') as f:
            folder_encodings = json.load(f)
    except FileNotFoundError:
        folder_encodings = {}

    try:
        new_image = face_recognition.load_image_file(image_path)
        new_encodings = face_recognition.face_encodings(new_image)  # Get all encodings
    except IndexError:
        print(f"Warning: No face detected in {image_path}. Skipping.")
        return

    for new_encoding in new_encodings:  # Iterate through each detected face
        best_match_folder = None
        best_match_distance = float('inf')

        # Check if the face matches any existing folder
        for folder, encoded_image in folder_encodings.items():
            distance = face_recognition.face_distance([encoded_image], new_encoding)[0]
            if distance < best_match_distance:
                best_match_distance = distance
                best_match_folder = folder

        # If a match is found, copy the image to the corresponding folder
        if best_match_folder and best_match_distance < 0.6:  # Adjust threshold 
            folder_path = os.path.join(directory, best_match_folder)
        else:
            # Create a new folder for a new face
            while True:
                new_folder = str(random.randint(10000, 99999))
                folder_path = os.path.join(directory, new_folder)
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                    folder_encodings[new_folder] = new_encoding.tolist()
                    break

        # Copy the image to the folder
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(folder_path, image_filename)
        shutil.copy2(image_path, new_image_path)  # Copy the image to the matching/new folder

    # Save the updated encodings back to the JSON file
    with open(paging_file, 'w') as f:
        json.dump(folder_encodings, f)

folder_to_pick_from = "assets/image_data/archive/pull"
folder_to_compare_from = "assets/image_data/archive/push"

for item_name in os.listdir(folder_to_pick_from):
    # Example usage:
    item_path = os.path.join(folder_to_pick_from, item_name)
    organize_images(item_path) 

### This will create a file and put in related images path 

In [3]:
import os
import random
import json
import face_recognition

def organize_images(image_path, directory="assets/image_data/archive/push", paging_file="assets/image_data/archive/push/image_index.json"):
    """
    Organizes images into JSON files based on similarity to existing images,
    using face encodings for comparison. Handles multiple faces per image.

    Instead of copying images, stores their names and paths in separate JSON files for each face.

    Args:
        image_path (str): Path to the new image to be organized.
        directory (str): Base directory for storing image paths. 
                                     Defaults to "assets/image_data/archive/push".
        paging_file (str): Name of the JSON file to store 
                                      folder-encoding mappings. 
                                      Defaults to "assets/image_data/archive/push/image_index.json".
    """

    if not os.path.exists(directory):
        os.makedirs(directory)

    try:
        with open(paging_file, 'r') as f:
            folder_encodings = json.load(f)
    except FileNotFoundError:
        folder_encodings = {}

    try:
        new_image = face_recognition.load_image_file(image_path)
        new_encodings = face_recognition.face_encodings(new_image)  # Get all encodings
    except IndexError:
        print(f"Warning: No face detected in {image_path}. Skipping.")
        return

    for new_encoding in new_encodings:  # Iterate through each detected face
        best_match_file = None
        best_match_distance = float('inf')

        # Check if the face matches any existing encoding
        for file_name, encoded_image in folder_encodings.items():
            distance = face_recognition.face_distance([encoded_image], new_encoding)[0]
            if distance < best_match_distance:
                best_match_distance = distance
                best_match_file = file_name

        # If a match is found, use the corresponding file
        if best_match_file and best_match_distance < 0.6:  # Adjust threshold 
            file_name = best_match_file
        else:
            # Create a new file for a new face
            while True:
                new_file = str(random.randint(10000, 99999)) + ".json"
                file_name = new_file
                if file_name not in folder_encodings:
                    folder_encodings[file_name] = new_encoding.tolist()
                    break

        # Load or create the JSON file for the face
        file_path = os.path.join(directory, file_name)
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                image_paths = json.load(f)
        else:
            image_paths = {}

        # Add the image name and path to the JSON file
        image_name = os.path.basename(image_path)
        image_paths[image_name] = image_path

        # Write the updated image paths back to the JSON file
        with open(file_path, 'w') as f:
            json.dump(image_paths, f, indent=4)

    # Save the updated encodings back to the paging JSON file
    with open(paging_file, 'w') as f:
        json.dump(folder_encodings, f)

# Example usage for processing all images in the folder_to_pick_from
folder_to_pick_from = "assets/image_data/archive/pull"
for item_name in os.listdir(folder_to_pick_from):
    item_path = os.path.join(folder_to_pick_from, item_name)
    organize_images(item_path)
